# This notebook enables recreation of the list of ICU stays used in our paper, after gaining access to MIMIC-III. 
### It does not contain the full preprocessing methods, only those necessary for ICUSTAY filtration.
Parts of this notebook were inspired by https://github.com/sebbarb/time_aware_attention, the accompanying repository to the paper: 
**Barbieri, Sebastiano et al. “Benchmarking Deep Learning Architectures for Predicting Readmission to the ICU and Describing Patients-at-Risk.” Scientific Reports 10 (2019)**

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import warnings
tqdm.pandas()
warnings.filterwarnings('ignore')

In [2]:
# insert your own path here. It should contain all the tables you downloaded from MIMIC-III in csv format
mimic_dir = ""

# Demographics
### This is the initial filtering, based on patient demographics and details of the stay.

In [3]:
print('Load ICU stays...')
dtype = {'SUBJECT_ID': 'int32',
       'HADM_ID': 'int32',
       'ICUSTAY_ID': 'int32',
       'INTIME': 'str',
       'OUTTIME': 'str',
       'LOS': 'float32'}
parse_dates = ['INTIME', 'OUTTIME']
icustays = pd.read_csv(mimic_dir + 'ICUSTAYS.csv', usecols=dtype.keys(), dtype=dtype, parse_dates=parse_dates)
print('Number of ICUSTAYS Initially:', icustays.ICUSTAY_ID.nunique())

print('-----------------------------------------')

print('Load patients...')
dtype = {'SUBJECT_ID': 'int32',
       'GENDER': 'str',
       'DOB': 'str',
       'DOD': 'str'}
parse_dates = ['DOB', 'DOD']
patients = pd.read_csv(mimic_dir + 'PATIENTS.csv', usecols=dtype.keys(), dtype=dtype, parse_dates=parse_dates)  

# Adjust shifted DOBs for older patients (median imputation)
old_patient = patients['DOB'].dt.year < 2000
date_offset = pd.DateOffset(years=(300-91), days=(-0.4*365))
patients['DOB'][old_patient] = patients['DOB'][old_patient].apply(lambda x: x + date_offset)

print('-----------------------------------------')
# Link icustays and patients tables
print('Link icustays and patients tables...')
icu_pat = pd.merge(icustays, patients, how='inner', on='SUBJECT_ID')
icu_pat.sort_values(by=['SUBJECT_ID', 'OUTTIME'], ascending=[True, False], inplace=True)

# Exclude icu stays during which patient died
icu_pat = icu_pat[~(icu_pat['DOD'] <= icu_pat['OUTTIME'])]

# Determine number of icu discharges in the last 365 days
print('Compute number of recent admissions...')
icu_pat['NUM_RECENT_ADMISSIONS'] = 0
for name, group in tqdm(icu_pat.groupby(['SUBJECT_ID'])):
    for index, row in group.iterrows():
        days_diff = (row['OUTTIME']-group['OUTTIME']).dt.days
        icu_pat.at[index, 'NUM_RECENT_ADMISSIONS'] = len(group[(days_diff > 0) & (days_diff <= 365)])

# Create age variable and exclude patients < 18 y.o.
icu_pat['AGE'] = (icu_pat['OUTTIME'] - icu_pat['DOB']).dt.days/365.
icu_pat = icu_pat[icu_pat['AGE'] >= 18]

# Time to next admission (discharge to admission!)
icu_pat['DAYS_TO_NEXT'] = (icu_pat.groupby(['SUBJECT_ID']).shift(1)['INTIME'] - icu_pat['OUTTIME']).dt.days

# Add early readmission flag (less than 30 days after discharge)
icu_pat['POSITIVE'] = (icu_pat['DAYS_TO_NEXT'] <= 30)

# Add early death flag (less than 30 days after discharge)
early_death = ((icu_pat['DOD'] - icu_pat['OUTTIME']).dt.days <= 30)

# Censor negative patients who died within less than 30 days after discharge (no chance of readmission)
icu_pat = icu_pat[icu_pat['POSITIVE'] | ~early_death]

# remove icustays that are part of a sequence-i.e, the hospital already knows them. Remove extreme LOS
icu_pat_first_yearly = icu_pat[icu_pat.NUM_RECENT_ADMISSIONS == 0]
icu_pat_first_yearly_normal = icu_pat_first_yearly[(icu_pat_first_yearly.LOS >= 1) & (icu_pat_first_yearly.LOS <= 30)]
icu_pat = icu_pat_first_yearly_normal

# Clean up
icu_pat.drop(columns=['DOB', 'DOD', 'DAYS_TO_NEXT'], inplace=True)

print('-----------------------------------------')
print('Total nulls per column (should be zero): ')
print(icu_pat.isnull().sum())

print('-----------------------------------------')
icu_pat.sort_values(by='ICUSTAY_ID', ascending=True, inplace=True)

print('Number of ICUSTAYS remaining:', icu_pat.ICUSTAY_ID.nunique())

Load ICU stays...
Number of ICUSTAYS Initially: 61532
-----------------------------------------
Load patients...
-----------------------------------------
Link icustays and patients tables...
Compute number of recent admissions...


100%|███████████████████████████████████████████████████████████████████████████| 43126/43126 [01:42<00:00, 420.09it/s]


-----------------------------------------
Total nulls per column (should be zero): 
SUBJECT_ID               0
HADM_ID                  0
ICUSTAY_ID               0
INTIME                   0
OUTTIME                  0
LOS                      0
GENDER                   0
NUM_RECENT_ADMISSIONS    0
AGE                      0
POSITIVE                 0
dtype: int64
-----------------------------------------
Number of ICUSTAYS remaining: 30339


# Discharge Notes
### Here we remove ICU stays without discharge notes, or with conflicting notes.
Notes are linked to addmissions, we need to convert them to ICU stays.

In [4]:
print('Getting patients with notes')
notes = pd.read_csv(mimic_dir + 'NOTEEVENTS.csv')
rel_notes = notes[(notes.HADM_ID.isin(set(icu_pat.HADM_ID))) & (notes.CATEGORY == 'Discharge summary') & (notes.DESCRIPTION=='Report')]
merged_notes = rel_notes.merge(icu_pat[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID']], on=['HADM_ID', 'SUBJECT_ID'])
simple_case = merged_notes.HADM_ID.value_counts()[merged_notes.HADM_ID.value_counts() == 1].index
merged_notes = merged_notes[merged_notes.HADM_ID.isin(simple_case)]

print('-----------------------------------------')
print('Removing ICU stays without notes:')
icu_pat = icu_pat[icu_pat.ICUSTAY_ID.isin(set(merged_notes.ICUSTAY_ID))]

print('Number of ICUSTAYS remaining:', icu_pat.ICUSTAY_ID.nunique())

Getting patients with notes
-----------------------------------------
Removing ICU stays without notes:
Number of ICUSTAYS remaining: 28678


# Creating Labs and Charts data
### Before we can select ICU stays with sufficient data, we need to create the relevant tables for our use case

In [5]:
lab_measurments = ['HGB_BLOOD', 'PLT_BLOOD', 'GLUCOSE_BLOOD', 'PH_BLOOD', 'PO2_BLOOD',
                   'PCO2_BLOOD', 'SODIUM_BLOOD', 'CHLORIDE_BLOOD', 'PHOSPHATE_BLOOD',
                   'CREATININE_BLOOD', 'UREA_BLOOD', 'WBC_BLOOD']
chart_measurments = ['GCS_TOTAL', 'HEART_RATE', 'BP', 'BODY_TEMP', 'RESPIRATORY_RATE']

def item_to_concept_map(item_id):
    #1
    if item_id in [50811, 51222, 814, 220228]:
        return 'HGB_BLOOD'
    #2
    elif item_id in [51266, 51265, 828, 227457]:
        return 'PLT_BLOOD'
    #3
    elif item_id in [50809, 50931, 51478, 807, 811, 1529, 220621, 225664, 226537]:
        return 'GLUCOSE_BLOOD'
    #4
    elif item_id in [51491, 50820, 780, 223830, 1126]:
        return 'PH_BLOOD'
    #5
    elif item_id in [50821]:
        return 'PO2_BLOOD'
    #6
    elif item_id in [50818]:
        return 'PCO2_BLOOD'
    #7
    elif item_id in [50824, 50983, 220645, 837, 1536]:
        return 'SODIUM_BLOOD'
    #8
    elif item_id in [50806, 50902, 788, 1523, 220602]:
        return 'CHLORIDE_BLOOD'
    #9
    elif item_id in [50863, 50970]:
        return 'PHOSPHATE_BLOOD'
    #10
    elif item_id in [50912, 1525, 220615, 791]:
        return 'CREATININE_BLOOD'
    #11
    elif item_id in [51006]:
        return 'UREA_BLOOD'
    #12
    elif item_id in [861, 1127, 1542, 220546]:
        return 'WBC_BLOOD'
    #13
    elif item_id in [198, 226755]:
        return 'GCS_TOTAL'
    #13
    elif item_id in [184, 220739, 226756, 227011]:
        return 'GCS_EYE_OPENING'
    #13
    elif item_id in [723, 223900, 226758, 227014]:
        return 'GCS_VERBAL_RESPONSE'
    #13
    elif item_id in [454, 223901, 226757, 227012]:
        return 'GCS_MOTOR_RESPONSE'
    #14
    elif item_id in [211, 220045, 227018]:
        return 'HEART_RATE'
    #15
    elif item_id in [52, 443, 456, 2293, 2294, 2647, 3312, 3314, 3320, 6590, 6702, 6927, 7620, 220052, 220181, 225312]:
        return 'BP'
    #16
    elif item_id in [676, 677, 678, 679, 3652, 3654, 6643, 223761, 223762, 226778, 227054]:
        return 'BODY_TEMP'
    #17
    elif item_id in [614, 615, 618, 619, 651, 653, 1884, 3603, 6749, 7884, 8113, 220210, 224422, 224688, 224689, 224690, 226774, 227050]:
        return 'RESPIRATORY_RATE'
    else:
        return None


In [6]:
parse_dates = ['CHARTTIME']
print('Loading Lab Data')
lab_data = pd.read_csv(mimic_dir + 'LABEVENTS.csv', parse_dates=parse_dates)[['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM']]

print('Keeping only relevant patients')
lab_data = lab_data[lab_data.HADM_ID.isin(set(icu_pat.HADM_ID))]

print('Converting lab_events admissions to ICUSTAY_IDS')
lab_data = lab_data.merge(icu_pat, on=['SUBJECT_ID', 'HADM_ID'], how='inner')
lab_data = lab_data[(lab_data.CHARTTIME >= lab_data.INTIME) & (lab_data.CHARTTIME <= lab_data.OUTTIME)]
lab_data = lab_data[['ICUSTAY_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM']]

print('Converting ITEMID to CONCEPT')
lab_data.dropna(inplace=True)
lab_data['CONCEPT'] = lab_data.ITEMID.progress_apply(lambda x: item_to_concept_map(x)) 
lab_data.dropna(inplace=True)

Loading Lab Data
Keeping only relevant patients
Converting lab_events admissions to ICUSTAY_IDS
Converting ITEMID to CONCEPT


100%|████████████████████████████████████████████████████████████████████| 5666019/5666019 [00:14<00:00, 400867.16it/s]


In [7]:
print('Loading Chart Data')
chart_data = pd.read_csv(mimic_dir + 'CHARTEVENTS.csv', parse_dates=parse_dates)[['ICUSTAY_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM']]

print('Keeping only relevant patients')
chart_data = chart_data[chart_data.ICUSTAY_ID.isin(set(icu_pat.ICUSTAY_ID))]

print('Converting ITEMID to CONCEPT')
chart_data.dropna(inplace=True)
chart_data['CONCEPT'] = chart_data.ITEMID.progress_apply(lambda x: item_to_concept_map(x)) 
chart_data.dropna(inplace=True)

Loading Chart Data
Keeping only relevant patients
Converting ITEMID to CONCEPT


100%|██████████████████████████████████████████████████████████████████| 16985809/16985809 [00:37<00:00, 447737.37it/s]


We need to clean the data in multiple ways to fully understand what is available: <br>
1)If we have two conflicting measurements at the same time, we keep only the maximum. <br>
2)We have data that should be in lab_data that appears in chart_data. We merge the two sources, and resolve conflicts by giving priority to lab_data. <br>
3)We combine all three GCS concepts. Only if all three or the total is provided, we count it as having the measurement. 

Keeping only maximum of each timestamp

In [8]:
lab_data = lab_data.groupby(['ICUSTAY_ID', 'CHARTTIME', 'CONCEPT']).max().reset_index()
chart_data = chart_data.groupby(['ICUSTAY_ID', 'CHARTTIME', 'CONCEPT']).max().reset_index()

Transferring lab tests from chart_data back to lab_data

In [9]:
temp_chart_lab = chart_data[chart_data.CONCEPT.isin(lab_measurments)]
existing_lab = lab_data.set_index(['ICUSTAY_ID', 'CHARTTIME', 'CONCEPT']).index
temp_chart_lab = temp_chart_lab.set_index(['ICUSTAY_ID', 'CHARTTIME', 'CONCEPT'])
missing_lab = temp_chart_lab.index.difference(existing_lab)
temp_chart_lab = temp_chart_lab.loc[missing_lab, :].reset_index()
lab_data = pd.concat([lab_data, temp_chart_lab.reset_index()])\
        .groupby(['ICUSTAY_ID', 'CHARTTIME', 'CONCEPT']).max().reset_index().drop(columns=['index'])
chart_data = chart_data[~chart_data.CONCEPT.isin(lab_measurments)]

Combining GCS measurements

In [10]:
chart_data.sort_values(by=['ICUSTAY_ID', 'CHARTTIME'], ascending=[True, False], inplace=True)
# Compute GCS total if not available
rows_gcs = (chart_data['CONCEPT'] == 'GCS_EYE_OPENING') | (chart_data['CONCEPT'] == 'GCS_VERBAL_RESPONSE') | (
            chart_data['CONCEPT'] == 'GCS_MOTOR_RESPONSE') | (chart_data['CONCEPT'] == 'GCS_TOTAL' )
chart_data_gcs = chart_data[rows_gcs]
chart_data_gcs = chart_data_gcs.pivot_table(index=['ICUSTAY_ID', 'CHARTTIME'], columns='CONCEPT',
                                    values='VALUENUM')
chart_data_gcs = chart_data_gcs.rename_axis(None, axis=1).reset_index()
null_gcs_total = chart_data_gcs['GCS_TOTAL'].isnull()
chart_data_gcs.loc[null_gcs_total, 'GCS_TOTAL'] = chart_data_gcs['GCS_EYE_OPENING'] + chart_data_gcs['GCS_VERBAL_RESPONSE'] + chart_data_gcs[
    'GCS_MOTOR_RESPONSE']
chart_data_gcs = chart_data_gcs.rename(columns={'GCS_TOTAL': 'VALUENUM'})
chart_data_gcs['CONCEPT'] = 'GCS_TOTAL'
chart_data_gcs.drop(columns=['GCS_EYE_OPENING', 'GCS_VERBAL_RESPONSE', 'GCS_MOTOR_RESPONSE'], inplace=True)

# Merge back with rest of the table
rows_others = ~rows_gcs 
chart_data = pd.concat([chart_data_gcs, chart_data[rows_others]], ignore_index=True, sort=False).drop(columns=['ITEMID'])
chart_data.sort_values(by=['ICUSTAY_ID', 'CHARTTIME'], ascending=[True, False], inplace=True)

## Remove stays with insufficient measurements
### We demand at least one from each lab test measurement, and five from each chart (high frequency) measurement.

Removing from lab_data

In [11]:
lab_data = lab_data[lab_data.CONCEPT.isin(lab_measurments)].dropna()
# we count the number of observations for each stay and each measurement, filling in zeros for stays with no
# observations for a measurement, then we take the minimum number of observations for each stay and remove stays
# with less than 5 observations
min_counts_labs = lab_data.groupby(['ICUSTAY_ID', 'CONCEPT']).count().unstack(fill_value=0).stack()['CHARTTIME'].groupby(level=0).min()
sufficient_stays_labs = min_counts_labs[min_counts_labs >= 1].index
len(set(sufficient_stays_labs))

15661

Removing from chart_data

In [12]:
chart_data = chart_data[chart_data.CONCEPT.isin(chart_measurments)].dropna()
min_counts_charts = chart_data.groupby(['ICUSTAY_ID', 'CONCEPT']).count().unstack(fill_value=0).stack()['CHARTTIME'].groupby(level=0).min()
sufficient_stays_charts = min_counts_charts[min_counts_charts >= 5].index
len(set(sufficient_stays_charts))

27519

In [13]:
sufficient_stays = set(sufficient_stays_labs).intersection(set(sufficient_stays_charts))
len(sufficient_stays)

15424

# Create final ICUSTAY set and print demographics

In [14]:
icu_pat = icu_pat[icu_pat.ICUSTAY_ID.isin(sufficient_stays)]

In [15]:
icu_pat['OVER_65'] = icu_pat.AGE.apply(lambda x: x>65)

In [16]:
print(icu_pat.SUBJECT_ID.nunique())

14837


In [17]:
print('Number of ICUSTAYS remaining:', icu_pat.ICUSTAY_ID.nunique())

Number of ICUSTAYS remaining: 15424


In [18]:
print(icu_pat.groupby(['GENDER', 'OVER_65', 'POSITIVE']).count()[['ICUSTAY_ID']])

                         ICUSTAY_ID
GENDER OVER_65 POSITIVE            
F      False   False           2659
               True             303
       True    False           3176
               True             433
M      False   False           4268
               True             485
       True    False           3569
               True             531


In [19]:
print(icu_pat.groupby('POSITIVE').count().ICUSTAY_ID,
      icu_pat.groupby('POSITIVE').count().ICUSTAY_ID/icu_pat.shape[0])

POSITIVE
False    13672
True      1752
Name: ICUSTAY_ID, dtype: int64 POSITIVE
False    0.886411
True     0.113589
Name: ICUSTAY_ID, dtype: float64


In [20]:
icu_pat.ICUSTAY_ID.to_csv(mimic_dir + 'list_of_stays.csv')